In [1]:
from SALib.sample import saltelli
from SALib.analyze import sobol
from SALib.test_functions import Ishigami
from math import *
import numpy as np
import time
from numba import njit
import ctypes

In [2]:
#Коэффициенты
a = 0.2
b = 0.4

In [3]:
#Придумаем какую-нибудь многомерную функцию
def evaluate_model(x):
    return cos(x[0]) + b * exp(x[1]) + a * x[2]**2 + sin(x[0]) * x[3]**3

сделаем сразу копию-заготовку, чтобы оценить производительность numba

In [4]:
@njit
def evaluate_model_njit(x):
    return cos(x[0]) + b * exp(x[1]) + a * x[2]**2 + sin(x[0]) * x[3]**3

In [5]:
problem = {
    'num_vars' : 4,
    'names' : ['x1', 'x2', 'x3', 'x4'],
    'bounds' : [[-5.0, 5.0],
                [-5.0, 5.0],
                [-5.0, 5.0],
                [-5.0, 5.0]
               ]
}

### NAIVE

In [6]:
for n in [1000, 10000, 100000, 1000000]:
    print('n = ' + str(n))
    start1 = time.time()
    param_values = saltelli.sample(problem, n)
    print("samples generation took %s seconds" %(time.time() - start1))

    Y = np.zeros(param_values.shape[0])

    start2 = time.time()
    for i, X in enumerate(param_values):
        Y[i] = evaluate_model(X)
    print("model evaluation took %s seconds" %(time.time() - start2))

    start3 = time.time()
    S = sobol.analyze(problem, Y)
    print("sobol analyzing took %s seconds" %(time.time() - start3))
    print('time spent: ' + str(time.time() - start1))
    print('_________________________________________')

n = 1000
samples generation took 0.06200885772705078 seconds
model evaluation took 0.023009300231933594 seconds
sobol analyzing took 0.08800005912780762 seconds
time spent: 0.173018217086792
_________________________________________
n = 10000
samples generation took 0.38803911209106445 seconds
model evaluation took 0.22400546073913574 seconds
sobol analyzing took 1.125655174255371 seconds
time spent: 1.7408041954040527
_________________________________________
n = 100000
samples generation took 3.5757288932800293 seconds
model evaluation took 2.236370325088501 seconds
sobol analyzing took 12.386244297027588 seconds
time spent: 18.199350357055664
_________________________________________
n = 1000000
samples generation took 38.1824471950531 seconds
model evaluation took 22.305063724517822 seconds
sobol analyzing took 201.18292713165283 seconds
time spent: 261.6723964214325
_________________________________________


### With NUMBA

In [7]:
for n in [1000, 10000, 100000, 1000000]:
    print('n = ' + str(n))
    start1 = time.time()
    param_values = saltelli.sample(problem, n)
    print("samples generation took %s seconds" %(time.time() - start1))

    Y = np.zeros(param_values.shape[0])

    start2 = time.time()
    for i, X in enumerate(param_values):
        Y[i] = evaluate_model_njit(X)
    print("model evaluation took %s seconds" %(time.time() - start2))

    start3 = time.time()
    S = sobol.analyze(problem, Y)
    print("sobol analyzing took %s seconds" %(time.time() - start3))
    print('time spent: ' + str(time.time() - start1))
    print('_________________________________________')

n = 1000
samples generation took 0.062027931213378906 seconds
model evaluation took 0.22422051429748535 seconds
sobol analyzing took 0.09000110626220703 seconds
time spent: 0.38321971893310547
_________________________________________
n = 10000
samples generation took 0.38291406631469727 seconds
model evaluation took 0.05500078201293945 seconds
sobol analyzing took 1.1458327770233154 seconds
time spent: 1.5837476253509521
_________________________________________
n = 100000
samples generation took 3.924820899963379 seconds
model evaluation took 0.5322489738464355 seconds
sobol analyzing took 13.342687845230103 seconds
time spent: 17.799757719039917
_________________________________________
n = 1000000
samples generation took 37.72104787826538 seconds
model evaluation took 5.46831488609314 seconds
sobol analyzing took 196.224454164505 seconds
time spent: 239.41481828689575
_________________________________________


In [8]:
# first-order indices
print(S['S1'])

[0.00049283 0.10671535 0.00168328 0.00020866]


In [9]:
# total indices for each variables
print(S['ST'])

[0.89155584 0.10676564 0.00168302 0.89124519]


In [10]:
# second order indices
print("x1-x2: ", S['S2'][0, 1])
print("x1-x3: ", S['S2'][0, 2])
print("x1-x4: ", S['S2'][0, 3])
print("x2-x3: ", S['S2'][1, 2])
print("x2-x4: ", S['S2'][1, 3])
print("x3-x4: ", S['S2'][2, 3])

x1-x2:  1.3383421256354944e-05
x1-x3:  -3.9213709952956415e-05
x1-x4:  0.890995103298507
x2-x3:  5.843712795765678e-05
x2-x4:  4.889035115416971e-05
x3-x4:  3.815437371321238e-07


### CTYPES

In [11]:
# Загрузка библиотеки
libfunc = ctypes.CDLL('libfunc.so')

In [12]:
# Указываем, что функция возвращает double
libfunc.evaluate_model.restype = ctypes.c_double
# Указываем, что функция принимает аргументы double
libfunc.evaluate_model.argtypes = [ctypes.c_double, ctypes.c_double, ctypes.c_double, ctypes.c_double]

In [13]:
for n in [1000, 10000, 100000, 1000000]:
    print('n = ' + str(n))
    start1 = time.time()
    param_values = saltelli.sample(problem, n)
    print("samples generation took %s seconds" %(time.time() - start1))

    Y = np.zeros(param_values.shape[0])

    start2 = time.time()
    for i, X in enumerate(param_values):
        Y[i] = libfunc.evaluate_model(X[0], X[1], X[2], X[3])
    print("model evaluation took %s seconds" %(time.time() - start2))

    start3 = time.time()
    S = sobol.analyze(problem, Y)
    print("sobol analyzing took %s seconds" %(time.time() - start3))
    print('time spent: ' + str(time.time() - start1))
    print('_________________________________________')

n = 1000
samples generation took 0.06255722045898438 seconds
model evaluation took 0.045999765396118164 seconds
sobol analyzing took 0.09200096130371094 seconds
time spent: 0.2085583209991455
_________________________________________
n = 10000
samples generation took 0.3830382823944092 seconds
model evaluation took 0.1699967384338379 seconds
sobol analyzing took 1.1081700325012207 seconds
time spent: 1.6612050533294678
_________________________________________
n = 100000
samples generation took 3.640841245651245 seconds
model evaluation took 1.846695899963379 seconds
sobol analyzing took 12.944727420806885 seconds
time spent: 18.43526530265808
_________________________________________
n = 1000000
samples generation took 37.25505089759827 seconds
model evaluation took 17.300273895263672 seconds
sobol analyzing took 202.77235507965088 seconds
time spent: 257.32959246635437
_________________________________________


### Parallel

In [14]:
from threading import Thread

In [15]:
def par_func(t):
    for i, X in enumerate(param_values[int(t*n/10) : int((t + 1)*n/10)]):
        Y[int(t*n/10) : int((t + 1)*n/10)] = evaluate_model(X)

In [18]:
for n in [1000, 10000, 100000, 1000000]:
    print('n = ' + str(n))
    start1 = time.time()
    param_values = saltelli.sample(problem, n)
    print("samples generation took %s seconds" %(time.time() - start1))

    Y = np.zeros(param_values.shape[0])

    start2 = time.time()
    threads = {}
    for t in range(10):
        threads.update({'t' + str(t) : Thread(target=par_func, args=[t])})
    for t in range(10):
        threads['t' + str(t)].start()
    for t in range(10):
        threads['t' + str(t)].join()
    print("model evaluation took %s seconds" %(time.time() - start2))

    start3 = time.time()
    S = sobol.analyze(problem, Y)
    print("sobol analyzing took %s seconds" %(time.time() - start3))
    print('time spent: ' + str(time.time() - start1))
    print('_________________________________________')

n = 1000
samples generation took 0.08600211143493652 seconds
model evaluation took 0.006964683532714844 seconds
sobol analyzing took 0.0890355110168457 seconds
time spent: 0.19100236892700195
_________________________________________
n = 10000
samples generation took 0.3785512447357178 seconds
model evaluation took 0.1024789810180664 seconds
sobol analyzing took 1.1391465663909912 seconds
time spent: 1.6205863952636719
_________________________________________
n = 100000
samples generation took 3.744311809539795 seconds
model evaluation took 1.0009465217590332 seconds
sobol analyzing took 12.680296421051025 seconds
time spent: 17.425934314727783
_________________________________________
n = 1000000
samples generation took 36.197612047195435 seconds
model evaluation took 12.934038162231445 seconds
sobol analyzing took 200.1975064277649 seconds
time spent: 249.33229207992554
_________________________________________
